In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import requests
from jsonmerge import merge
from pprint import pprint
import pandas as pd
from pandas.io.json import json_normalize
import json
import io

pd.set_option('display.max_columns', None)

# YELP DATASETS

In [7]:
df_biz = pd.read_json("yelp_academic_dataset_business.json", lines = True)
df_covid = pd.read_json("yelp_academic_dataset_covid_features.json", lines = True)

In [8]:
df_biz.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,10913 Bailey Rd,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...",f9NumwFMBDn751xgFiRbNA,"Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...",Cornelius,"{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'...",1,35.462724,-80.852612,The Range At Lake Norman,28031,36,3.5,NC
1,"8880 E Via Linda, Ste 107","{'GoodForKids': 'True', 'ByAppointmentOnly': '...",Yzvjg0SayhoZgCljUJRF9Q,"Health & Medical, Fitness & Instruction, Yoga,...",Scottsdale,None,1,33.569404,-111.890264,"Carlos Santo, NMD",85258,4,5.0,AZ
2,3554 Rue Notre-Dame O,None,XNoUzKckATkOD1hP6vghZg,"Pets, Pet Services, Pet Groomers",Montreal,None,1,45.479984,-73.580070,Felinus,H4C 1P4,5,5.0,QC
3,1015 Sharp Cir,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",6OAZjbxqM5ol29BuHsil3w,"Hardware Stores, Home Services, Building Suppl...",North Las Vegas,"{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ...",0,36.219728,-115.127725,Nevada House of Hose,89030,3,2.5,NV
4,4827 E Downing Cir,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",51M2Kk903DFYI6gnB5I6SQ,"Home Services, Plumbing, Electricians, Handyma...",Mesa,"{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '...",1,33.428065,-111.726648,USE MY GUY SERVICES LLC,85205,26,4.5,AZ


In [9]:
df_covid

,Call To Action enabled,Covid Banner,Grubhub enabled,Request a Quote Enabled,Temporary Closed Until,Virtual Services Offered,business_id,delivery or takeout,highlights
0,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,9kXRUIkwdDtnAPO6tVo51g,FALSE,FALSE
1,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,H6D5HOTfMjrZt7r1EObZ1g,FALSE,FALSE
2,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FYddq7fUtzobZcw4jOJgVA,FALSE,FALSE
3,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,c75jLTjlgA9q3gImLEGT6w,FALSE,FALSE
4,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,YfzPiY50h_10Sjlg3mnNWQ,FALSE,FALSE
5,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,HqDRvA64BpGtjnzf9beE4Q,FALSE,FALSE
6,TRUE,We are closely monitoring COVID-19 and adjusti...,FALSE,TRUE,FALSE,offers_virtual_tours,4voKpEbJhrhLv3FR__JgLA,FALSE,"[{""identifier"":""remote_services_during_covid_1..."
7,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,S-xZzaHeMyMcqf66GAC4wA,FALSE,FALSE
8,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,c9sQmFjO8NnaLAFOVqWivw,FALSE,FALSE
9,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,64ZUb-llBHrnOcN_o1wA_A,TRUE,FALSE


### CLEAN & TRANSFORM DATA

In [17]:
# Join the two DataFrame according to 'business_id'
df_yelp = pd.merge(df_biz, df_covid, on = 'business_id')
df_yelp.shape

(209795, 22)

In [11]:
# Filter out Las Vegas
df_lv = df_yelp[df_yelp.city == 'Las Vegas']

In [12]:
# Filter out just 'Restaurants' from 'categories' column
df_lv_food = df_lv[df_lv.categories.str.contains("Restaurants",na=False)]

In [13]:
# Drop the 'hours' columns
df_lv_food = df_lv_food.drop(columns = ['hours'])

In [14]:
# Filter out only restaurants that are open
df_lv_food = df_lv_food[df_lv_food.is_open == 1]

In [15]:
# Then drop the 'is_open' column
df_lv_food = df_lv_food.drop(columns = ['is_open'])

In [16]:
df_lv_food.shape

(4386, 20)

In [19]:
# Parsing a dictionary in a pandas dataframe cell into new row cells (new columns)
df_lv_food__attributes = df_lv_food.attributes.apply(pd.Series)
# Take only the relevant columns
df_lv_food__attributes_useful = df_lv_food__attributes[[
    'RestaurantsDelivery',
    'RestaurantsTakeOut',
    'RestaurantsReservations',
    'OutdoorSeating',
    'RestaurantsGoodForGroups',
    'DriveThru',
    'ByAppointmentOnly'
    ]]
df_lv_food__attributes_useful.head()

,RestaurantsDelivery,RestaurantsTakeOut,RestaurantsReservations,OutdoorSeating,RestaurantsGoodForGroups,DriveThru,ByAppointmentOnly
86,False,True,False,NaN,NaN,NaN,NaN
101,False,True,False,False,True,True,NaN
157,True,True,False,False,True,False,NaN
238,False,True,False,False,True,NaN,False
246,True,True,True,False,True,NaN,False


In [20]:
df = pd.concat([df_lv_food,df_lv_food__attributes_useful], axis = 1)
df = df.drop(columns = ['attributes'])
df

,address,business_id,categories,city,latitude,longitude,name,postal_code,review_count,stars,state,Call To Action enabled,Covid Banner,Grubhub enabled,Request a Quote Enabled,Temporary Closed Until,Virtual Services Offered,delivery or takeout,highlights,RestaurantsDelivery,RestaurantsTakeOut,RestaurantsReservations,OutdoorSeating,RestaurantsGoodForGroups,DriveThru,ByAppointmentOnly
86,,OWkS1FXNJbozn-qPg3LWxg,"Food, Food Trucks, Restaurants, Pizza",Las Vegas,36.128561,-115.171130,Mama Napoli Pizza,89109,11,4.5,NV,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,False,True,False,NaN,NaN,NaN,NaN
101,8033 N Durango Dr,MTx-Zdl_KcU_z9G832XAjg,"Fast Food, Restaurants",Las Vegas,36.307495,-115.279059,Taco Bell,89131,37,2.5,NV,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,False,True,False,False,True,True,NaN
157,"6125 W Tropicana Ave, Ste F",Yr_w9lakJrKMyEG_hI6zbA,"Pizza, Salad, Burgers, Restaurants",Las Vegas,36.099361,-115.226636,Fat Moe's Pizza & Wings,89103,141,4.0,NV,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,True,True,False,False,True,False,NaN
238,3759 E Desert Inn Rd,AN0bWhisCf6LN9eHZ7DQ3w,"Restaurants, Italian",Las Vegas,36.129178,-115.092483,Los Olivos Ristorante,89121,222,5.0,NV,FALSE,We are open 7 days a week from 2:30 pm to 9:00...,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,False,True,False,False,True,NaN,False
246,"5115 Spring Mountain Rd, Ste 203",AtD6B83S4Mbmq0t7iDnUVA,"Restaurants, Specialty Food, Japanese, Sushi B...",Las Vegas,36.125569,-115.210911,Veggie House,89146,1142,4.5,NV,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,True,True,True,False,True,NaN,False
271,4205 Spring Mountain Rd,bjqwzEr9fU1keOcJFOZqWA,"Local Flavor, Restaurants, Food Stands, Street...",Las Vegas,36.125722,-115.196053,Chinese New Year Celebration,89102,13,4.0,NV,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,NaN,NaN,NaN,True,NaN,NaN,NaN
309,"7729 S Rainbow Blvd, Ste 9B",oUX2bYbqjqST-urKbOHG6w,"Sandwiches, Shaved Ice, Coffee & Tea, Desserts...",Las Vegas,36.047942,-115.244167,Loftti Cafe,89139,284,4.5,NV,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,True,True,False,True,True,NaN,NaN
316,5020 Blue Diamond Rd,FiW6w5nmhlUoJAyNofb4jg,"Coffee & Tea, Food Stands, Food, Restaurants, ...",Las Vegas,36.032122,-115.210267,Fruits and Roots,89139,106,4.5,NV,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,NaN,True,NaN,True,NaN,NaN,NaN
365,5757 Wayne Newton Blvd,ZsKWULhwwB61RHzCrb1i9A,"Restaurants, Mexican",Las Vegas,36.080542,-115.146459,Blue Burrito Grille,89119,31,2.5,NV,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,False,True,False,False,True,NaN,NaN
451,3950 Las Vegas Blvd So,c1JoHp602zilpDU_57DsMg,"Restaurants, Cafes, Food, Internet Cafes, Coff...",Las Vegas,36.093270,-115.175966,Starbucks,89119,57,2.0,NV,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,True,True,NaN,False,NaN,NaN,NaN


In [ ]:
df.to_csv("/Users/mucongl/Downloads/large_datasets/yelp_cleaned_for_covid_db.csv")

## FOURSQUARE API

In [6]:
import json, requests
url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
client_id='MIHKP5UNU0XOMNO4JLI0L41RCBX5VSS0M2FQNMEIATKQ3VVM',
client_secret='1HY2OUGSAIWTBKSPZQYQEL1NP5B25EJKSZZAX2R1N0V5ZFQY',
v='20180323',
near='Las Vegas',
query='food',
limit=500
)

resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

In [8]:
data

{'meta': {'code': 200, 'requestId': '5faaa3549d297d7a46d7f432'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'geocode': {'what': '',
   'where': 'las vegas',
   'center': {'lat': 36.17497, 'lng': -115.13722},
   'displayString': 'Las Vegas, NV, United States',
   'cc': 'US',
   'geometry': {'bounds': {'ne': {'lat': 36.32397712011264,
      'lng': -114.89639282226568},
     'sw': {'lat': 35.9285367031886, 'lng': -115.35507202148438}}},
   'slug': 'las-vegas-nevada',
   'longId': '72057594043434892'},
  'headerLocation': 'Las Vegas',
  'headerFullLocation': 'Las Vegas',
  'headerLocationGranularity': 'city',
  'query': 'food',
  'totalResults': 201,
  'suggestedBounds': {'ne': {'lat': 36.285812032773855,
    'lng': -115.01900112501446},
   'sw': {'lat': 35.97765731174918, 'lng': -115.34824870011889}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
  

In [18]:
result = requests.get(url, params=params).json()
result['response']['groups'][0]

{'type': 'Recommended Places',
 'name': 'recommended',
 'items': [{'reasons': {'count': 0,
    'items': [{'summary': 'This spot is popular',
      'type': 'general',
      'reasonName': 'globalInteractionReason'}]},
   'venue': {'id': '4c3b6e58f5c0b713c9ea8029',
    'name': 'Red Rock Feast Buffet',
    'contact': {},
    'location': {'address': '11011 W Charleston Blvd',
     'crossStreet': 'at Red Rock Casino Resort Spa',
     'lat': 36.156917960451715,
     'lng': -115.33275350968441,
     'labeledLatLngs': [{'label': 'display',
       'lat': 36.156917960451715,
       'lng': -115.33275350968441}],
     'postalCode': '89135',
     'cc': 'US',
     'city': 'Las Vegas',
     'state': 'NV',
     'country': 'United States',
     'formattedAddress': ['11011 W Charleston Blvd (at Red Rock Casino Resort Spa)',
      'Las Vegas, NV 89135',
      'United States']},
    'categories': [{'id': '52e81612bcbc57f1066b79f4',
      'name': 'Buffet',
      'pluralName': 'Buffets',
      'shortName': '

In [10]:
df_foursquare = json_normalize(data['response']['groups'][0]['items'])
df_foursquare

,reasons.count,reasons.items,referralId,venue.beenHere.count,venue.beenHere.lastCheckinExpiredAt,venue.beenHere.marked,venue.beenHere.unconfirmedCount,venue.categories,venue.delivery.id,venue.delivery.provider.icon.name,...,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.stats.checkinsCount,venue.stats.tipCount,venue.stats.usersCount,venue.stats.visitsCount,venue.venuePage.id,venue.verified
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c3b6e58f5c0b713c9ea8029-0,0,0,False,0,"[{'id': '52e81612bcbc57f1066b79f4', 'name': 'B...",NaN,NaN,...,NV,Red Rock Feast Buffet,0,[],0,0,0,0,NaN,True
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b5b3e5af964a520d1ed28e3-1,0,0,False,0,"[{'id': '4bf58dd8d48988d146941735', 'name': 'D...",1461773,/delivery_provider_grubhub_20180129.png,...,NV,Weiss Deli and Bakery,0,[],0,0,0,0,NaN,False
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b4173e0f964a520c2c625e3-2,0,0,False,0,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",NaN,NaN,...,NV,"Joe's Seafood, Prime Steak & Stone Crab",0,[],0,0,0,0,88110757,True
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-525a017b498e2e085ea6b924-3,0,0,False,0,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",1315819,/delivery_provider_grubhub_20180129.png,...,NV,Soho Japanese Restaurant,0,[],0,0,0,0,NaN,False
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ab918ccf964a520fc7d20e3-4,0,0,False,0,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",NaN,NaN,...,NV,In-N-Out Burger,0,[],0,0,0,0,NaN,True
5,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4a96e2c6f964a520622720e3-5,0,0,False,0,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",2036692,/delivery_provider_grubhub_20180129.png,...,NV,Egg & I,0,[],0,0,0,0,NaN,True
6,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-545908e7498eb65c7e72c8b2-6,0,0,False,0,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",350442,/delivery_provider_grubhub_20180129.png,...,NV,Blueberry Hill Family Restaurant,0,[],0,0,0,0,NaN,False
7,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-5918d0802aff31013ad02271-7,0,0,False,0,"[{'id': '54135bf5e4b08f3d2429dfe5', 'name': 'A...",NaN,NaN,...,NV,Black Bear Diner,0,[],0,0,0,0,NaN,False
8,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-514d187ce4b02a09b32e2e30-8,0,0,False,0,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",NaN,NaN,...,NV,Juan's Flaming Fajita and Cantina,0,[],0,0,0,0,NaN,False
9,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-58cab383966e55657504045e-9,0,0,False,0,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",2415546,/delivery_provider_grubhub_20180129.png,...,NV,Chick-fil-A,0,[],0,0,0,0,NaN,True


In [13]:
df_foursquare['venue.categories'].head()

0    [{'id': '52e81612bcbc57f1066b79f4', 'name': 'B...
1    [{'id': '4bf58dd8d48988d146941735', 'name': 'D...
2    [{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...
3    [{'id': '4bf58dd8d48988d111941735', 'name': 'J...
4    [{'id': '4bf58dd8d48988d16e941735', 'name': 'F...
Name: venue.categories, dtype: object

In [14]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories']
nearby_venues = nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

NameError: name 'nearby_venues' is not defined

# ZOMATO API

In [11]:
URL = "https://developers.zomato.com/api/v2.1/cities?q=las%20vegas"
header = {"User-agent": "curl/7.43.0", "Accept": "application/json", "user_key": "5978814818e9908f86073519a08ca500"}

response = requests.get(URL, headers=header)


In [12]:
zomato_api = '5978814818e9908f86073519a08ca500'

In [13]:
def get_location_details(query):

    headers = {
        'Accept': 'application/json',
        'user-key': zomato_api,
    }
    params = (
        ('query', query),
    )

    response = requests.get('https://developers.zomato.com/api/v2.1/locations', headers=headers, params=params)
    data = response.json()

    for loc in data['location_suggestions']:
        loc_id = loc['entity_id']
        loc_type = loc['entity_type']

    return loc_id, loc_type

In [14]:
def get_restaurants(ent_id, ent_type):

    headers = {
        'Accept': 'application/json',
        'user-key': zomato_api,
    }

    params = (
        ('entity_id', ent_id),
        ('entity_type', ent_type),
    )

    response = requests.get('https://developers.zomato.com/api/v2.1/search', headers=headers, params=params)

    return response.json()

In [15]:
entity_id, entity_type = get_location_details("Las Vegas")
print(entity_id)
print(entity_type)

282
city


In [16]:
zomato_data = get_restaurants(entity_id, entity_type)
zomato_data

{'results_found': 5401,
 'results_start': 0,
 'results_shown': 20,
 'restaurants': [{'restaurant': {'R': {'res_id': 16979446,
     'is_grocery_store': False,
     'has_menu_status': {'delivery': -1, 'takeaway': -1}},
    'apikey': '5978814818e9908f86073519a08ca500',
    'id': '16979446',
    'name': 'Mon Ami Gabi',
    'url': 'https://www.zomato.com/las-vegas/mon-ami-gabi-las-vegas?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1',
    'location': {'address': '3655 Las Vegas Boulevard South 89109',
     'locality': 'The Strip',
     'city': 'Las Vegas',
     'city_id': 282,
     'latitude': '36.1121300000',
     'longitude': '-115.1694200000',
     'zipcode': '89109',
     'country_id': 216,
     'locality_verbose': 'The Strip, Las Vegas'},
    'switch_to_order_menu': 0,
    'cuisines': 'Breakfast, French',
    'timings': '7 AM to 10 PM (Mon-Thu, Sun), 7 AM to 11 PM (Fri-Sat)',
    'average_cost_for_two': 70,
    'price_range': 4,
    'currency': '$',
    'highlights': ['Lunc

In [17]:
json_normalize(zomato_data['restaurants']).head()

,restaurant.R.has_menu_status.delivery,restaurant.R.has_menu_status.takeaway,restaurant.R.is_grocery_store,restaurant.R.res_id,restaurant.all_reviews.reviews,restaurant.all_reviews_count,restaurant.apikey,restaurant.average_cost_for_two,restaurant.book_again_url,restaurant.book_form_web_view_url,restaurant.cuisines,restaurant.currency,restaurant.deeplink,restaurant.establishment,restaurant.establishment_types,restaurant.events_url,restaurant.featured_image,restaurant.has_online_delivery,restaurant.has_table_booking,restaurant.highlights,restaurant.id,restaurant.include_bogo_offers,restaurant.is_book_form_web_view,restaurant.is_delivering_now,restaurant.is_table_reservation_supported,restaurant.is_zomato_book_res,restaurant.location.address,restaurant.location.city,restaurant.location.city_id,restaurant.location.country_id,restaurant.location.latitude,restaurant.location.locality,restaurant.location.locality_verbose,restaurant.location.longitude,restaurant.location.zipcode,restaurant.menu_url,restaurant.mezzo_provider,restaurant.name,restaurant.offers,restaurant.opentable_support,restaurant.phone_numbers,restaurant.photo_count,restaurant.photos_url,restaurant.price_range,restaurant.store_type,restaurant.switch_to_order_menu,restaurant.thumb,restaurant.timings,restaurant.url,restaurant.user_rating.aggregate_rating,restaurant.user_rating.rating_color,restaurant.user_rating.rating_obj.bg_color.tint,restaurant.user_rating.rating_obj.bg_color.type,restaurant.user_rating.rating_obj.title.text,restaurant.user_rating.rating_text,restaurant.user_rating.votes
0,-1,-1,False,16979446,"[{'review': []}, {'review': []}, {'review': []...",319,5978814818e9908f86073519a08ca500,70,,,"Breakfast, French",$,zomato://restaurant/16979446,[],[],https://www.zomato.com/las-vegas/mon-ami-gabi-...,https://b.zmtcdn.com/data/res_imagery/16979446...,0,0,"[Lunch, Serves Alcohol, Breakfast, Dinner, Out...",16979446,True,0,0,0,0,3655 Las Vegas Boulevard South 89109,Las Vegas,282,216,36.1121300000,The Strip,"The Strip, Las Vegas",-115.1694200000,89109,https://www.zomato.com/las-vegas/mon-ami-gabi-...,OTHER,Mon Ami Gabi,[],0,(702) 944-4224,241,https://www.zomato.com/las-vegas/mon-ami-gabi-...,4,,0,https://b.zmtcdn.com/data/res_imagery/16979446...,"7 AM to 10 PM (Mon-Thu, Sun), 7 AM to 11 PM (F...",https://www.zomato.com/las-vegas/mon-ami-gabi-...,4.6,3F7E00,800,lime,4.6,Excellent,1214
1,-1,-1,False,16982906,"[{'review': []}, {'review': []}, {'review': []...",285,5978814818e9908f86073519a08ca500,40,,,"Burger, Fast Food",$,zomato://restaurant/16982906,[Casual Dining],[],https://www.zomato.com/las-vegas/gordon-ramsay...,https://b.zmtcdn.com/data/res_imagery/16982906...,0,0,"[Cash, No Alcohol Available, Dinner, Debit Car...",16982906,True,0,0,0,0,3667 South Las Vegas Boulevard 89109,Las Vegas,282,216,36.1106106000,The Strip,"The Strip, Las Vegas",-115.1722684000,89109,https://www.zomato.com/las-vegas/gordon-ramsay...,OTHER,Gordon Ramsay Burger,[],0,"(702) 785-5555, (702) 785-5462",416,https://www.zomato.com/las-vegas/gordon-ramsay...,3,,0,https://b.zmtcdn.com/data/res_imagery/16982906...,"10:30 AM to 12 Midnight (Mon-Fri), 12 Midnight...",https://www.zomato.com/las-vegas/gordon-ramsay...,4.5,3F7E00,700,lime,4.5,Excellent,608
2,-1,-1,False,16977639,"[{'review': []}, {'review': []}, {'review': []...",175,5978814818e9908f86073519a08ca500,70,,,French,$,zomato://restaurant/16977639,[Fine Dining],[],https://www.zomato.com/las-vegas/bouchon-bistr...,https://b.zmtcdn.com/data/res_imagery/16977639...,0,0,"[Credit Card, Breakfast, Lunch, Serves Alcohol...",16977639,True,0,0,0,0,"The Venetian, 3355 Las Vegas Boulevard South, ...",Las Vegas,282,216,36.1210000000,The Strip,"The Strip, Las Vegas",-115.1702000000,89109,https://www.zomato.com/las-vegas/bouchon-bistr...,OTHER,Bouchon Bistro - The Venetian,[],0,(702) 414-6200,169,https://www.zomato.com/las-vegas/bouchon-bistr...,4,,0,https://b.zmtcdn.com/data/res_imagery/16977639...,"7 AM to 1 PM, 5 PM to 10 PM (Mon-Fri),8 AM to ...",h

In [18]:
zomato_df = json_normalize(zomato_data['restaurants'])
cols = ['restaurant.name', 'restaurant.R.res_id','restaurant.location.address', 'restaurant.all_reviews_count',  'restaurant.user_rating.aggregate_rating', 'restaurant.user_rating.votes' ]
zomato_relevant1 = zomato_df[cols]
zomato_relevant1

,restaurant.name,restaurant.R.res_id,restaurant.location.address,restaurant.all_reviews_count,restaurant.user_rating.aggregate_rating,restaurant.user_rating.votes
0,Mon Ami Gabi,16979446,3655 Las Vegas Boulevard South 89109,319,4.6,1214
1,Gordon Ramsay Burger,16982906,3667 South Las Vegas Boulevard 89109,285,4.5,608
2,Bouchon Bistro - The Venetian,16977639,"The Venetian, 3355 Las Vegas Boulevard South, ...",175,4.6,693
3,Lotus of Siam,16979203,"953 E Sahara Ave, Las Vegas 89104",157,4.6,606
4,Mesa Grill,16979381,3570 South Las Vegas Boulevard 89109,222,4.3,812
5,The Buffet at Bellagio,16981060,"3600 Las Vegas Blvd S, Las Vegas 89109",167,4.0,709
6,Burger Bar - Mandalay Bay,16977697,"Mandalay Bay, 3930 South Las Vegas Boulevard, ...",147,4.2,715
7,Hash House a Go Go,16981709,"3535 Las Vegas Blvd S, Las Vegas 89109",153,4.1,475
8,Earl of Sandwich,16978241,3667 Las Vegas Boulevard South 89109,113,4.8,437
9,Hash House a Go Go,16978652,"6800 W Sahara Ave, Las Vegas 89146",125,4.4,626


In [19]:
def get_restaurants2(ent_id, ent_type, start_count):

    headers = {
        'Accept': 'application/json',
        'user-key': zomato_api,
    }

    params = (
        ('entity_id', ent_id),
        ('entity_type', ent_type),
        ('start', start_count)
    )

    response = requests.get('https://developers.zomato.com/api/v2.1/search', headers=headers, params=params)

    return response.json()

In [19]:
zomato_data2 = get_restaurants2(entity_id, entity_type, 20)
zomato_df2 = json_normalize(zomato_data2['restaurants'])
zomato_relevant2 = zomato_df2[cols]
zomato_relevant2

,restaurant.name,restaurant.R.res_id,restaurant.location.address,restaurant.all_reviews_count,restaurant.user_rating.aggregate_rating,restaurant.user_rating.votes
0,Spago,16980318,3500 Las Vegas Boulevard. South 89109,74,4.7,354
1,L'Atelier de Joël Robuchon,16979023,"3799 Las Vegas Boulevard S, Las Vegas 89109",75,4.9,224
2,Bachi Burger,16982003,"470 East Windmill Lane, Suite 100, South Las V...",81,4.4,259
3,Jean-Philippe Pâtisserie,16978840,"3600 Las Vegas Boulevard S, Las Vegas 89109",83,4.9,237
4,Hot N Juicy Crawfish,16981028,"4810 Spring Mountain Road, Las Vegas 89102",49,4.6,261
5,Lucille's Smokehouse Bar-B-Que,16979209,"2245 Village Walk Dr, Henderson 89052",68,4.1,406
6,Julian Serrano,16981685,"3730 Las Vegas Boulevard S, Las Vegas 89109",99,4.6,280
7,Raku,16981099,5030 Spring Mountain Road 89146,54,4.7,153
8,Gordon Ramsay Steak,16982201,3655 South Las Vegas Boulevard 89109,143,4.6,280
9,Tao Asian Bistro,16980571,"3355 Las Vegas Boulevard S, Las Vegas 89109",82,4.1,354


In [20]:
zomato_data3 = get_restaurants2(entity_id, entity_type, 40)
zomato_df3 = json_normalize(zomato_data3['restaurants'])
zomato_relevant3 = zomato_df3[cols]

In [21]:
zomato_data4 = get_restaurants2(entity_id, entity_type, 60)
zomato_df4 = json_normalize(zomato_data4['restaurants'])
zomato_relevant4 = zomato_df4[cols]

In [21]:
zomato_data5 = get_restaurants2(entity_id, entity_type, 80)
zomato_df5 = json_normalize(zomato_data5['restaurants'])
zomato_relevant5 = zomato_df5[cols]

In [23]:
zomato_data7 = get_restaurants2(entity_id, entity_type, 100)
zomato_df7 = json_normalize(zomato_data7['restaurants'])
zomato_relevant7 = zomato_df7[cols]
zomato_relevant7

KeyError: "None of [Index(['restaurant.name', 'restaurant.R.res_id', 'restaurant.location.address',\n       'restaurant.all_reviews_count',\n       'restaurant.user_rating.aggregate_rating',\n       'restaurant.user_rating.votes'],\n      dtype='object')] are in the [columns]"

In [27]:
zomato_concat = [zomato_relevant1, zomato_relevant2, zomato_relevant3, zomato_relevant4, zomato_relevant5]
zomato_100 = pd.concat(zomato_concat)
zomato_100

,restaurant.name,restaurant.R.res_id,restaurant.location.address,restaurant.all_reviews_count,restaurant.user_rating.aggregate_rating,restaurant.user_rating.votes
0,Mon Ami Gabi,16979446,3655 Las Vegas Boulevard South 89109,319,4.6,1214
1,Gordon Ramsay Burger,16982906,3667 South Las Vegas Boulevard 89109,285,4.5,608
2,Bouchon Bistro - The Venetian,16977639,"The Venetian, 3355 Las Vegas Boulevard South, ...",175,4.6,693
3,Lotus of Siam,16979203,"953 E Sahara Ave, Las Vegas 89104",157,4.6,606
4,Mesa Grill,16979381,3570 South Las Vegas Boulevard 89109,222,4.3,812
5,The Buffet at Bellagio,16981060,"3600 Las Vegas Blvd S, Las Vegas 89109",167,4.0,709
6,Burger Bar - Mandalay Bay,16977697,"Mandalay Bay, 3930 South Las Vegas Boulevard, ...",147,4.2,715
7,Hash House a Go Go,16981709,"3535 Las Vegas Blvd S, Las Vegas 89109",153,4.1,475
8,Earl of Sandwich,16978241,3667 Las Vegas Boulevard South 89109,113,4.8,437
9,Hash House a Go Go,16978652,"6800 W Sahara Ave, Las Vegas 89146",125,4.4,626


In [29]:
zomato_100.to_csv(r'C:\Users\willm\OneDrive\Documents\MSCA\31012_Data_Engineering\Final_Project\Datasets\Yelp\zomato_100.csv')

# Nevada COVID-19 Case Data

In [2]:
url="http://covidtracking.com/api/states/daily.csv"
s=requests.get(url).content

In [3]:
covid_df = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [4]:
# format
covid_df = pd.read_csv(io.StringIO(s.decode('utf-8')))
covid_df['date'] =  pd.to_datetime(covid_df['date'], format='%Y%m%d')
covid_df.drop(['dateChecked'],axis=1,inplace=True)
covid_df['state']=covid_df['state'].apply(str)
covid_df.fillna(value=-1, inplace=True)

In [5]:
# filter to NV only
nv_covid_df = covid_df[covid_df['state'] == 'NV']

In [6]:
nv_covid_df

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,recovered,dataQualityGrade,lastUpdateEt,dateModified,checkTimeEt,death,hospitalized,totalTestsViral,positiveTestsViral,negativeTestsViral,positiveCasesViral,deathConfirmed,deathProbable,totalTestEncountersViral,totalTestsPeopleViral,totalTestsAntibody,positiveTestsAntibody,negativeTestsAntibody,totalTestsPeopleAntibody,positiveTestsPeopleAntibody,negativeTestsPeopleAntibody,totalTestsPeopleAntigen,positiveTestsPeopleAntigen,totalTestsAntigen,positiveTestsAntigen,fips,positiveIncrease,negativeIncrease,total,totalTestResultsIncrease,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
36,2020-11-28,NV,149229.0,-1.0,823887.0,-1.0,totalTestEncountersViral,1622816.0,1460.0,-1.0,317.0,-1.0,162.0,-1.0,-1.0,A,11/28/2020 00:00,2020-11-28T00:00:00Z,11/27 19:00,2119.0,-1.0,1154583.0,-1.0,-1.0,149229.0,-1.0,-1.0,1622816.0,973116.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,32,2912,4459,973116,15811,973116,24,0,d3aa4b5eb4305e87bd84575043277ff69f0b5908,0,0,0,0,0,-1.0
92,2020-11-27,NV,146317.0,-1.0,819428.0,-1.0,totalTestEncountersViral,1607005.0,1440.0,-1.0,306.0,-1.0,162.0,-1.0,-1.0,A,11/27/2020 00:00,2020-11-27T00:00:00Z,11/26 19:00,2095.0,-1.0,1154583.0,-1.0,-1.0,146317.0,-1.0,-1.0,1607005.0,965745.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,32,1536,5016,965745,14090,965745,2,0,1de96bad828d359b5d43c5c4c8a49e5b03094ebd,0,0,0,0,0,-1.0
148,2020-11-26,NV,144781.0,-1.0,814412.0,-1.0,totalTestEncountersViral,1592915.0,1440.0,-1.0,306.0,-1.0,162.0,-1.0,-1.0,A,11/26/2020 00:00,2020-11-26T00:00:00Z,11/25 19:00,2093.0,-1.0,1154583.0,-1.0,-1.0,144781.0,-1.0,-1.0,1592915.0,959193.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,32,2542,5935,959193,18658,959193,22,0,48042195744948c30b7350060ba01e01c651c89e,0,0,0,0,0,-1.0
204,2020-11-25,NV,142239.0,-1.0,808477.0,-1.0,totalTestEncountersViral,1574257.0,1414.0,-1.0,312.0,-1.0,169.0,-1.0,-1.0,A,11/25/2020 18:55,2020-11-25T18:55:00Z,11/25 13:55,2071.0,-1.0,1154583.0,-1.0,-1.0,142239.0,-1.0,-1.0,1574257.0,950716.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,32,3159,3831,950716,15631,950716,24,0,aa93d0255b5dc34148c2aff133b06c7f2889fd62,0,0,0,0,0,-1.0
260,2020-11-24,NV,139080.0,-1.0,804646.0,-1.0,totalTestEncountersViral,1558626.0,1399.0,-1.0,296.0,-1.0,161.0,-1.0,-1.0,A,11/24/2020 00:00,2020-11-24T00:00:00Z,11/23 19:00,2047.0,-1.0,1154583.0,-1.0,-1.0,139080.0,-1.0,-1.0,1558626.0,943726.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,32,2853,7285,943726,22550,943726,24,0,3ead006c2757f5a1a4ae4c3554f6385c4e008bbb,0,0,0,0,0,-1.0
316,2020-11-23,NV,136227.0,-1.0,797361.0,-1.0,totalTestEncountersViral,1536076.0,1274.0,-1.0,278.0,-1.0,163.0,-1.0,-1.0,A,11/23/2020 00:00,2020-11-23T00:00:00Z,11/22 19:00,2023.0,-1.0,1154583.0,-1.0,-1.0,136227.0,-1.0,-1.0,1536076.0,933588.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,32,2339,5435,933588,15531,933588,6,0,a1f46693cacfcdd926da979f387686a421b8d9c1,0,0,0,0,0,-1.0
372,2020-11-22,NV,133888.0,-1.0,791926.0,-1.0,totalTestEncountersViral,1520545.0,1273.0,-1.0,266.0,-1.0,152.0,-1.0,-1.0,A,11/22/2020 00:00,2020-11-22T00:00:00Z,11/21 19:00,2017.0,-1.0,1154583.0,-1.0,-1.0,133888.0,-1.0,-1.0,1520545.0,925814.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,32,2155,4280,925814,13481,925814,6,0,5a2596ff12f5510d930807b6d2b180af23068432,0,0,0,0,0,-1.0
428,2020-11-21,NV,131733.0,-1.0,787646.0,-1.0,totalTestEncountersViral,1507064.0,1273.0,-1.0,266.0,-1.0,152.0,-1.0,-1.0,A,11/21/2020 00:00,2020-11-21T00:00:00Z,11/20 19:00,2011.0,-1.0,1154583.0,-1.0,-1.0,131733.0,-1.0,-1.0,1507064.0,919379.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,32,2019,10966,919379,28901,919379,29,0,8b403288903ddb57421a4640783dde166c0fb46a,0,0,0,0,0,-1.0
484,2020-11-20,NV,129714.0,-1.0,776680.0,-

In [7]:
rel_cols = ['date',
'state',
'positive',
'negative',
'totalTestResults',
'hospitalizedCurrently',
'inIcuCurrently',
'onVentilatorCurrently',
'death',
'lastUpdateEt',
'dateModified']
nv_covid_df_rel = nv_covid_df[rel_cols]
nv_covid_df_rel

,date,state,positive,negative,totalTestResults,hospitalizedCurrently,inIcuCurrently,onVentilatorCurrently,death,lastUpdateEt,dateModified
36,2020-11-28,NV,149229.0,823887.0,1622816.0,1460.0,317.0,162.0,2119.0,11/28/2020 00:00,2020-11-28T00:00:00Z
92,2020-11-27,NV,146317.0,819428.0,1607005.0,1440.0,306.0,162.0,2095.0,11/27/2020 00:00,2020-11-27T00:00:00Z
148,2020-11-26,NV,144781.0,814412.0,1592915.0,1440.0,306.0,162.0,2093.0,11/26/2020 00:00,2020-11-26T00:00:00Z
204,2020-11-25,NV,142239.0,808477.0,1574257.0,1414.0,312.0,169.0,2071.0,11/25/2020 18:55,2020-11-25T18:55:00Z
260,2020-11-24,NV,139080.0,804646.0,1558626.0,1399.0,296.0,161.0,2047.0,11/24/2020 00:00,2020-11-24T00:00:00Z
316,2020-11-23,NV,136227.0,797361.0,1536076.0,1274.0,278.0,163.0,2023.0,11/23/2020 00:00,2020-11-23T00:00:00Z
372,2020-11-22,NV,133888.0,791926.0,1520545.0,1273.0,266.0,152.0,2017.0,11/22/2020 00:00,2020-11-22T00:00:00Z
428,2020-11-21,NV,131733.0,787646.0,1507064.0,1273.0,266.0,152.0,2011.0,11/21/2020 00:00,2020-11-21T00:00:00Z
484,2020-11-20,NV,129714.0,776680.0,1478163.0,1283.0,271.0,151.0,1982.0,11/20/2020 00:00,2020-11-20T00:00:00Z
540,2020-11-19,NV,127875.0,775298.0,1471764.0,1288.0,261.0,150.0,1953.0,11/19/2020 00:00,2020-11-19T00:00:00Z


In [8]:
nv_covid_df_rel.to_csv(r'C:\Users\willm\OneDrive\Documents\MSCA\31012 - Data Engineering\Final_Project\Datasets\Yelp\NV_COVIDdata_1122.csv')

In [9]:
nv_covid_df_rel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269 entries, 36 to 14963
Data columns (total 11 columns):
date                     269 non-null datetime64[ns]
state                    269 non-null object
positive                 269 non-null float64
negative                 269 non-null float64
totalTestResults         269 non-null float64
hospitalizedCurrently    269 non-null float64
inIcuCurrently           269 non-null float64
onVentilatorCurrently    269 non-null float64
death                    269 non-null float64
lastUpdateEt             269 non-null object
dateModified             269 non-null object
dtypes: datetime64[ns](1), float64(7), object(3)
memory usage: 25.2+ KB
